# Hadoop

- Examples From Video Lecture 

##  Demo HDFS

```
# From windows, connect to the Linux instance
PS:>  docker compose exec hive-server bash

# Basic Commands
$ hdfs dfs –command
OR
$ Hadoop fs –command

# what is the current directory?
$Hadoop fs –ls
 # its /user/root

# make it
$ Hadoop fs –mkdir /user/root

# make a folder
$ Hadoop fs –mkdir /user/root/grades

# upload to Hadoop
$ Hadoop fs –put /datasets/grades/*.tsv  /user/root/grades/

### View on  Web… look at those block sizes.


# copy a file
hadoop fs -cp grades/fall2015.tsv grades/foo.tsv

# combine with cat
Hadoop fs –cat grades/*

# make a file with getmerge – a useful command when a reducer produces more than one fgile
Hadoop fs –getmerge grades/* allgrades.tsv

```

## Demo Connecting to Hive

```
# From windows connect to the hive server
PS:> docker-compose exec hive-server bash

# From linux start the hive client
$ beeline -u jdbc:hive2://localhost:10000/default

# now you are in the Hive client
beeline> Show databases

beeline> Show tables

beeline> Create database

```

## Demo Hive Internal Managed  Tables
```

beeline> !sh hadoop fs -ls /user/root/grades

beeline> !sh hadoop fs -cat /user/root/grades/*

beeline> create table grades_internal ( year INT, semster STRING, course STRING, credits INT, grade STRING) row format delimited fields terminated by ‘\t’;

beeline> show tables;

beeline> select * from grades_internal;

# LOAD DATA
beeline> load data inpath ‘/user/root/grades/*.tsv’ overwrite into table grades_internal;

beeline> select * from grades_internal;

beeline> select sum(credits), year from grades_internal group by year;

beeline> drop table grades_internal;

# data is gone
beeline> !sh hadoop fs -ls /user/hive/warehouse/grades_int


```

## Demo Hive External Tables

```
# re-load some grades into HDFS
beeline> !sh  hadoop fs -put /datasets/grades/fall2015.tsv /user/root/grades

beeline> !sh  hadoop fs -ls /user/root/grades


beeline> create external table grades_ext (
    year int,
    term string,
    course string,
    credits int,
    letter string
) row format delimited
fields terminated by '\t'
location '/user/root/grades';

beeline> select * from grades_ext;

# let’s “insert more data….
beeline> !sh  hadoop fs -put /datasets/grades/fall2016.tsv /user/root/grades


beeline> drop table grades_ext;

# data is still there
beeline> !sh hadoop fs -ls /user/cloudera/grades
```


## Demo of Spark Concepts

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
        .config("hive.metastore.uris", 
                "thrift://hive-metastore:9083") \
        .enableHiveSupport() \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR") # Keeps the noise down!!!

22/12/08 16:15:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/08 16:15:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/08 16:15:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
data = [
    (1, "Fido", "Dog","SPCA",1),
    (2, "Felix", "Cat", "SPCA",2),
    (3, "Rover", "Dog","SPCA",1)
]
cols = ["id","name","type","shelter","years_at_shelter"]
pets = spark.createDataFrame(data = data, schema = cols)
pets.show()

+---+-----+----+-------+----------------+
| id| name|type|shelter|years_at_shelter|
+---+-----+----+-------+----------------+
|  1| Fido| Dog|   SPCA|               1|
|  2|Felix| Cat|   SPCA|               2|
|  3|Rover| Dog|   SPCA|               1|
+---+-----+----+-------+----------------+



In [ ]:
#Filter then sort, then select
plan = pets.where( pets['years_at_shelter'] ==1).sort("name").select("name","type")
plan.explain()
plan.show()

In [ ]:
# select, filter then sort
plan = pets.select("name","type").where( pets['years_at_shelter'] ==1).sort("name")
plan.explain()
plan.show()

In [ ]:
plan1 = pets.select("name","type")
plan2 = plan1.where( pets['years_at_shelter'] ==1)
plan = plan2.sort("name")
plan.explain()
plan.show()

## Demo of Hive / Spark Integration

In [ ]:
spark.sql("drop table if exists default.grades")
spark.sql("""
create external table default.grades (
  year int,
  semester string,
  course string,
  credits int,
  grade string
) 
row format delimited 
fields terminated by '\t' 
location  'hdfs:///user/root/grades/*.tsv'
""")
spark.sql("select * from default.grades").show()

In [ ]:
spark.sql('''
select sum(credits), year, semester 
from default.grades 
group by year, semester
''').explain()

In [ ]:
# Hive Internal Table create stuff.

spark.sql("""drop table if exists default.department""")
spark.sql("""CREATE TABLE default.department(
department_id int ,
department_name string
)    
""")
spark.sql("""
INSERT INTO default.department values (101,"Oncology")    
""")
spark.sql("""
INSERT INTO default.department values (102,"Hematology")    
""")
spark.sql("SELECT * FROM default.department").show()

In [ ]:
spark.sql("create table default.department2 stored as avro as select * from default.department")

In [ ]:
spark.sql("select * from default.department2").show()